# Рекомендация тарифов

Имеются данные о поведении клиентов, которые уже перешли на эти тарифы «Смарт» или «Ультра». Необходимо построить модель для задачи классификации, которая выберет подходящий тариф. Следует построить модель с максимально большим значением accuracy и проверить на  тестовой выборке.

План действий:
1. Изучить данные.
2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.
3. Исследовать качество разных моделей, меняя гиперпараметры.
4. Проверьте качество модели на тестовой выборке.

## Откройте и изучите файл

In [1]:
import pandas
from sklearn.model_selection import train_test_split

import pandas as pd
import os
from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pandas.read_csv('/datasets/users_behavior.csv')

df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
df.shape

(3214, 5)

Датафрейм имеет 3214 строк и 5 столбцов.
Столбец "is_ultra" является целевым признаком. Необходимо создать модель, которая будет присваивать значение 1 пользователям, которым стоит предложить сменить тариф на "Ультра".
Поставлена задача классификации.

## Разбейте данные на выборки

In [4]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=1)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=1)

In [5]:
#Код ревьюера
df_train.shape, df_test.shape, df_valid .shape

((1928, 5), (643, 5), (643, 5))

Обучающая выборка 60%, валидационная и тестовая по 20%.

## Исследуйте модели

In [6]:
#переменные для признаков и целевого признака
features_train = df.drop(['is_ultra'], axis=1)
target_train = df['is_ultra']
features_valid = df.drop(['is_ultra'], axis=1)
target_valid = df['is_ultra']
features_test = df.drop(['is_ultra'], axis=1)
target_test = df['is_ultra']

In [7]:
#функция для подсчета точности модели на валидационной выборке
def show_accuracy_score(model):
    predictions_valid = model.predict(features_valid)
    return accuracy_score(target_valid, predictions_valid)

Применим 3 модели: дерево решений, случайный лес, логистическая регрессия.

Дерево решений

In [8]:
from sklearn.tree import DecisionTreeClassifier
for depth in range(1, 20, 2):
    model_tree = DecisionTreeClassifier(random_state=1, max_depth=depth,
            criterion='gini',
            min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, splitter='best')# модель с заданной глубиной дерева
    model_tree.fit(features_train, target_train) # обучуние модели
    print('depth =', depth, ' \ accuracy =', show_accuracy_score(model_tree))

depth = 1  \ accuracy = 0.752644679527069
depth = 3  \ accuracy = 0.798070939639079
depth = 5  \ accuracy = 0.8170504044803982
depth = 7  \ accuracy = 0.8397635345364033
depth = 9  \ accuracy = 0.8637212196639701
depth = 11  \ accuracy = 0.8855009334163036
depth = 13  \ accuracy = 0.9079029247044181
depth = 15  \ accuracy = 0.9327940261356565
depth = 17  \ accuracy = 0.9601742377100186
depth = 19  \ accuracy = 0.9816428126944617


Лучший результат при параметре глубины: depth = 19 \ accuracy = 0.9816428126944617

Случайный лес

In [9]:
for estim in range(10, 101, 10):
    model_forest = RandomForestClassifier(n_estimators=estim, max_depth=9, random_state=1,
            criterion='gini',
            min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0)
    model_forest.fit(features_train, target_train)
    print('n_estimators =', estim, ' / accuracy_score =', show_accuracy_score(model_forest))

n_estimators = 10  / accuracy_score = 0.8680771624144369
n_estimators = 20  / accuracy_score = 0.8708774113254512
n_estimators = 30  / accuracy_score = 0.8714996888612321
n_estimators = 40  / accuracy_score = 0.8736776602364654
n_estimators = 50  / accuracy_score = 0.8752333540759178
n_estimators = 60  / accuracy_score = 0.8746110765401369
n_estimators = 70  / accuracy_score = 0.8758556316116988
n_estimators = 80  / accuracy_score = 0.8752333540759178
n_estimators = 90  / accuracy_score = 0.8746110765401369
n_estimators = 100  / accuracy_score = 0.8746110765401369


Лучший результат при параметре количества деревьев и глубины 9(взяли с прошлого удачного показателя): depth = 9 / n_estimators = 70 / accuracy_score = 0.8758556316116988. Хуже, чем в дереве решений.

Логистическая регрессия

In [10]:
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
def logistic_regression(solver=solvers):
    model = LogisticRegression(solver=solver, random_state=1)
    model.fit(features_train, target_train)
    return show_accuracy_score(model)
for solver in solvers:
    print(solver, ': ', logistic_regression(solver=solver))

newton-cg :  0.7479775980087119
lbfgs :  0.7479775980087119
liblinear :  0.7423771001866832
sag :  0.6938394523957685
saga :  0.693528313627878


Лучший результат дают параметры newton-cg и lbfgs: 0.7479775980087119

Модели с параметрами дали лучший результат среди других параметров:

Decision Tree(depth = 9): accuracy =  0.9816428126944617,
Random Forest(depth = 9 / n_estimators = 70): 0.8758556316116988,
Logistic Regression(solver='newton-cg'): accuracy = 0.7479775980087119.

В итоге мы выбираем Случайный лес с параметром глубины 9. Дерево решений показывает почти идеальный результат, который вызывает подозрение, поэтому не учитываем.

## Проверьте модель на тестовой выборке

In [11]:
model = DecisionTreeClassifier(max_depth=9, random_state=1)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
accuracy_score(target_test, predictions_test)

0.8637212196639701

На тестовой выборке выбранная модель показывает результат близкий к валидационной, но не больше. Это означает отсутствие проблем с переобучением модели.

## Проверьте модели на адекватность

In [12]:
import numpy as np
from sklearn.dummy import DummyRegressor
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(features_train, target_train)
DummyRegressor()
dummy_regr.predict(features_train)
dummy_regr.score(features_train, target_train)


0.0

In [14]:
# Код ревьюера
from sklearn.dummy import DummyClassifier
dummy_cl = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_cl.fit(features_train, target_train)
dummy_cl.score(features_train, target_train)

0.693528313627878

Было проведено исследование, чтобы выявить модель для задачи классификации, которая выберет подходящий тариф для новых пользователей. Входные данные - данные о поведении клиентов, которые уже перешли на тарифы «Смарт» или «Ультра». Перед проведением исследования была поставлена задача - постройте модель с максимально большим значением accuracy.

Модели дали лучшие результаты:

Decision Tree(depth = 9): accuracy = 0.9816428126944617, 
Random Forest(depth = 9 / n_estimators = 70): 0.8758556316116988, 
Logistic Regression(solver='newton-cg'): accuracy = 0.7479775980087119.

Следует выбирать модель Случайный лес с параметром глубины 9, которая показала лучшее значение. Также на тестовой выборке данная модель показала отсутствие проблем с переобучением модели. 
